<a href="https://colab.research.google.com/github/kdhue0709/-1/blob/main/Qdrant_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://81d27b00-17bd-4597-aaad-bb7faf154027.us-east4-0.gcp.cloud.qdrant.io:6333
# sk-xxx

In [ ]:
!pip install langchain qdrant_client openai tiktoken

In [ ]:
from langchain.vectorstores import Qdrant
from langchain.embeddings.openai import OpenAIEmbeddings
import qdrant_client
import os

In [ ]:
# create a qdrant client
# create하기 전에 API key와 HOST를 변수에 저장
os.environ['QDRANT_HOST'] = 'https://81d27b00-17bd-4597-aaad-bb7faf154027.us-east4-0.gcp.cloud.qdrant.io:6333'
os.environ['QDRANT_API_KEY'] = 'qdrant_api_key_xxx'

# 클러스터에 연결을 허용
client = qdrant_client.QdrantClient(
    os.getenv("QDRANT_HOST"),
    api_key = os.getenv("QDRANT_API_KEY")
)


In [ ]:
# create collection
os.environ['QDRANT_COLLECTION_NAME'] = "collection_1"


vectors_config = qdrant_client.http.models.VectorParams(
    size=1536, # 768 for instuctor-x1, 1536 for openai. we have to be consistent with the same embeddings model for each collection
    distance=qdrant_client.http.models.Distance.COSINE
    )

client.recreate_collection(
    collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
    vectors_config = vectors_config,
)



True

In [ ]:
# create vector store

os.environ["OPENAI_API_KEY"] = "sk-xxx"

embeddings = OpenAIEmbeddings()

vector_store = Qdrant(
    client=client,
    collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
    embeddings=embeddings,
)

In [ ]:
# add documents to vector store

from langchain.text_splitter import CharacterTextSplitter

# 문장이 길지 않게 split
def get_chunks(text):
  text_splitter = CharacterTextSplitter(
      separator = "\n",
      chunk_size = 1000,
      chunk_overlap = 200,
      length_function = len
      )

  chunks = text_splitter.split_text(text)
  return chunks

with open('data.txt') as f:
  raw_text = f.read()

texts = get_chunks(raw_text)

vector_store.add_texts(texts)

print(texts)


['Title: "The Forgotten Letter"\nOnce upon a time in a quaint little town, there lived an elderly woman named Mrs. Thompson. She had spent most of her life in the same cozy house, surrounded by memories and the warmth of nostalgia. Mrs. Thompson was known for her kind heart and the delicious pies she baked for the neighborhood.\nOne day, as she was rummaging through her attic, Mrs. Thompson stumbled upon an old, dusty box. Curiosity got the better of her, and she opened it to find a stack of letters tied with a faded ribbon. These were letters she had written to her childhood friend, Sarah, many decades ago.\nAs she read through the letters, Mrs. Thompson was flooded with memories of their adventures, secrets, and dreams they had shared. However, something caught her eye—a letter she had never sent. It was a heartfelt apology for a misunderstanding that had led to a rift between them.', "Regret weighed heavily on Mrs. Thompson's heart as she realized she had never patched things up wit

In [ ]:
# plug vector store into retrieval chain

from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

qa = RetrievalQA.from_chain_type(
    llm = OpenAI(),
    chain_type = "stuff",
    retriever = vector_store.as_retriever()

)

In [ ]:
query = "How many friends are there and what are their names?"

response = qa.run(query)

print(response)

 There are two friends mentioned in the context, Mrs. Thompson and Sarah.
